In [133]:
from fugue_notebook import setup
import pandas as pd
import seaborn as sns
from typing import List, Any, Dict, Tuple, Iterable
import matplotlib.pyplot as plt
setup()

<IPython.core.display.Javascript object>

In [134]:
%%fsql 
df = LOAD "usa_names_2013.csv" (header = true, infer_schema = true)
# YIELD DATAFRAME AS df
PRINT


,state:str,gender:str,year:long,name:str,number:long
0,AR,F,2013,Abigail,122
1,AR,F,2013,Serenity,62
2,AR,F,2013,Lauren,39
3,AR,F,2013,Hannah,73
4,AR,F,2013,Victoria,44
5,AR,F,2013,Charlotte,37
6,AR,F,2013,Makayla,37
7,AZ,F,2013,Brooke,50
8,AZ,F,2013,Charlotte,184
9,AZ,F,2013,Grace,138


In [135]:
%%fsql
SELECT COUNT(*) AS cnt FROM df
PRINT

,cnt:long
0,92372


In [136]:
def concat(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    return pd.concat([df1, df2], axis=0).reset_index(drop = True)

In [137]:
def concat2(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    return pd.concat([df1.reset_index(drop = True), df2.reset_index(drop = True)], axis=1)

In [138]:
# schema: *, sex:str
def make_gender (data:Iterable[Dict[str, Any]]) -> Iterable[Dict[str, Any]]:
    for row in data:
        if row['gender'] == 'F':
            row['sex'] = 'Female'
        else :
            row['sex'] = 'Male' 
        yield row

In [139]:
%%fsql 
SELECT * FROM df 
TRANSFORM USING make_gender
YIELD DATAFRAME AS df_with_gender
PRINT

,state:str,gender:str,year:long,name:str,number:long,sex:str
0,AR,F,2013,Abigail,122,Female
1,AR,F,2013,Serenity,62,Female
2,AR,F,2013,Lauren,39,Female
3,AR,F,2013,Hannah,73,Female
4,AR,F,2013,Victoria,44,Female
5,AR,F,2013,Charlotte,37,Female
6,AR,F,2013,Makayla,37,Female
7,AZ,F,2013,Brooke,50,Female
8,AZ,F,2013,Charlotte,184,Female
9,AZ,F,2013,Grace,138,Female


In [140]:
%%fsql 
df_maxname_male = 
SELECT name, number 
FROM df
WHERE gender == 'M'  AND state = 'CA'
ORDER BY number DESC
LIMIT 1
PRINT

df_maxname_female =
SELECT name, number 
FROM df
WHERE gender == 'F'  AND state = 'CA'
ORDER BY number DESC
LIMIT 1

PROCESS df_maxname_female, df_maxname_male USING concat
PRINT

,name:str,number:long
1867,Jacob,2866


,name:str,number:long
0,Sophia,3442
1,Jacob,2866


In [141]:
%%fsql 
df_maxname_male = 
SELECT name, number ,sex
FROM df_with_gender
WHERE gender == 'M'  AND state = 'CA'
ORDER BY number DESC
LIMIT 10
YIELD DATAFRAME AS df_malemax_10

df_maxname_female =
SELECT name, number ,sex
FROM df_with_gender
WHERE gender == 'F'  AND state = 'CA'
ORDER BY number DESC
LIMIT 10
YIELD DATAFRAME AS df_femalemax_10


kkk = PROCESS df_maxname_female, df_maxname_male USING concat 
PRINT 20 ROWS 

,name:str,number:long,sex:str
0,Sophia,3442,Female
1,Isabella,2776,Female
2,Mia,2583,Female
3,Emma,2468,Female
4,Emily,2279,Female
5,Olivia,2141,Female
6,Sofia,1967,Female
7,Camila,1490,Female
8,Abigail,1485,Female
9,Samantha,1396,Female


In [142]:
%%fsql
df_max_axis1_f = SELECT name, number FROM df_femalemax_10
df_max_axis1_m = SELECT name AS name_, number AS number_ FROM df_malemax_10
kkk = PROCESS df_max_axis1_f,df_max_axis1_m  USING concat2
PRINT 10 ROWS FROM kkk


,name:str,number:long,name_:str,number_:long
0,Sophia,3442,Jacob,2866
1,Isabella,2776,Ethan,2649
2,Mia,2583,Daniel,2578
3,Emma,2468,Jayden,2575
4,Emily,2279,Matthew,2543
5,Olivia,2141,Noah,2534
6,Sofia,1967,Alexander,2364
7,Camila,1490,Anthony,2220
8,Abigail,1485,Nathan,2072
9,Samantha,1396,David,2024


In [143]:
%%fsql
SELECT gender, name, number, AVG(number) OVER(PARTITION BY state) AS number_avg
FROM df
WHERE state = 'CA'

YIELD DATAFRAME AS df_test


In [144]:
%%fsql
SELECT gender, name, number, number_avg 
FROM df_test
WHERE number < number_avg
ORDER BY number DESC
PRINT 10 ROWS



,gender:str,name:str,number:long,number_avg:double
251,F,Nathalia,62,62.920394
1382,M,Kash,62,62.920394
1640,M,Cristopher,62,62.920394
1861,M,Zaiden,62,62.920394
15842,F,Jayda,62,62.920394
15852,F,Aubrie,62,62.920394
16947,M,Marc,62,62.920394
17203,M,Bryant,62,62.920394
31740,F,Marie,62,62.920394
33422,M,Xavi,62,62.920394
